In [3]:
## Assignment 2
## NLP
## Test adnan

In [9]:
import numpy as np
import pandas as pd
import spacy
import string
nlp = spacy.load('en_core_web_sm')
import math

import time
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt

# Import stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [10]:
# Define stopwords list

stop = list(string.punctuation)
for x in stopwords.words('english'):
    stop.append(x)

'''
Add a couple of more stopwords which were observed manually
'''
for x in ['-PRON-','\n','...','..',"'d'","n't"]:
    stop.append(x)
    

In [88]:
# Define variables

corpus = 'books.txt'
encoding = 'ISO-8859-1'

alpha = 0.1 # Hyperparameter
beta = 0.1 # Hyperparameter
K = 30 # Nr of topics
M = 100000 # Corpus size
it = 100 # Nr of iterations


In [89]:
def clean_corpus(corpus:str, encoding:str, nr_tokens:int, stop:list) -> list():
    
    '''
    Only consider 100 000 tokens.
    Removes strings from stopword-list from corpus.
    Removes string with fewer characters than 3.
    Keeps lemmatized words.
    '''
    
    token_count = 0
    docs = list()
    docs_count = 0
    freqs = Counter()  
    
    with open(corpus, encoding = encoding) as f:
        
        while token_count < nr_tokens:
            
            doc = f.readline()
            docs_count += 1
            temp_doc = list()
            
            # Lemmatizion of tokens
            result = nlp(doc.lower())
            
            for token in result:
                if len(token.lemma_) > 2:
                    if token.lemma_ not in stop:
                        token_count += 1
                        freqs[token.lemma_] += 1
                        temp_doc.append(token.lemma_)
                        
            docs.append(temp_doc)

    print('Nr of tokens:' + str(token_count))
    print('Nr of docs:' + str(docs_count))
    
    return docs

def map_to_int(docs:list()) -> (list(), int, Counter()):
    
    '''
    Create bag of words from cleaned and smaller corpus.
    Associate each word in bag with an unique integer,
    ranging from 0 (most common word) to length of bag of words.
    Map each token in docs to the respective int. Return this list of list of ints.
    '''
    
    freqs = Counter()
    for doc in docs:
        for token in doc:
            freqs[token] += 1
    most_common = freqs.most_common()
    
    token_to_int = []
    for i in range(len(most_common)):
        token_to_int.append(most_common[i][0])

    # Get pairs of elements    
    mapping = zip(token_to_int, range(0,len(token_to_int)))
    
    # Make pairs into a dictionary
    vocab = dict(mapping)
        
    # Match token to int
    docs_int = list()
    
    for doc in docs:
        docs_int.append(list([vocab.get(x) for x in doc]))
        
    return docs_int, len(vocab), vocab

In [90]:
docs = clean_corpus(corpus=corpus, encoding=encoding, nr_tokens=M, stop=stop)
docs_mapped, vocab_size, vocab = map_to_int(docs)
n_docs = len(docs_mapped)

Nr of tokens:100010
Nr of docs:1246


# LDA STUFF

In [92]:
def init_lda(docs:list(), n_docs:int, n_topics:int, vocab_size:int) -> (np.matrix, np.matrix, np.matrix):
    
    # \Theta
    theta = np.zeros((n_docs,n_topics))
    for d in range(n_docs):
        theta[d] = np.random.dirichlet(alpha * np.ones(n_topics))

    # \phi
    phi = np.zeros((n_topics, vocab_size))
    for z in range(n_topics):
        phi[z] = np.random.dirichlet(beta*np.ones(vocab_size))
    
    # Word topic assignment
    Z_dj = list()
    for d in range(n_docs):
        Z_dj.append([0]*len(docs_mapped[d]))
    
    # Init wt by randomly assigning topics to each word in corpus
    for d in range(n_docs):
        for j in range(len(docs[d])):
            Z_dj[d][j] = np.random.randint(n_topics) # Generate random topic
        
    return theta, phi, Z_dj

def update(docs, Z_dj, n_docs, n_topics, vocab_size, theta, phi):
    
    # Counter for word j in document d
    nd = np.zeros((n_docs, n_topics))
    mk = np.zeros((n_topics, vocab_size))
    
    for d in range(n_docs):
        for j in range(len(docs[d])):
            nd[d][Z_dj[d][j]] += 1
            mk[Z_dj[d][j]][docs[d][j]] += 1
            
    return nd, mk

In [93]:
theta, phi, Z_dj = init_lda(docs_mapped, n_docs, K, vocab_size)

In [94]:
def gibbs(docs:list(), n_docs:int, n_topics:int, vocab_size:int, max_iterations:int, alpha:int, beta:int):

    start_time = time.time()
    theta, phi, Z_dj = init_lda(docs, n_docs=n_docs, n_topics=n_topics, vocab_size=vocab_size)
    print('Matrices initialized.')

    for it in range(max_iterations):
        #print('Iteration ' + str(it))
        
        nd, mk = update(docs, Z_dj, n_docs, n_topics, vocab_size, theta, phi)
        
        for d in range(n_docs):
            
            # Update \Theta
            theta[d] = np.random.dirichlet(alpha + nd[d])
            
        #print('Theta updated.')
        
        for k in range(n_topics):
            
            # Update \phi
            phi[k] = np.random.dirichlet(beta + mk[k])
            
        #print('phi updated.')
        
        for d in range(n_docs):
            for j in range(len(docs[d])):
                
                p_dw = np.exp(np.log(theta[d]) + np.log([row[docs[d][j]] for row in phi]))
                p_dw /= sum(p_dw)
                
                Z_dj[d][j] = np.random.multinomial(1, p_dw).argmax()
                
        if (it+1) % 10 == 0:
            print('Iteration ' + str(it+1))
                

    elapsed_time = time.time() - start_time
    print("Elapsed time: ", elapsed_time)
    
    return mk

In [95]:
mk = gibbs(docs_mapped, n_docs, K, vocab_size, it, alpha, beta)

Matrices initialized.
Iteration 10
Iteration 20
Iteration 30
Iteration 40
Iteration 50
Iteration 60
Iteration 70
Iteration 80
Iteration 90
Iteration 100
Elapsed time:  438.7033472061157


In [96]:
tops = list()
for k in range(K):
    order = np.array(mk[k]).argsort()[::-1][:10]
    dt = list()
    for x in order:
        for k,v in vocab.items():
            if x==v:
                dt.append(k)
    tops.append(dt)

In [97]:
tops

[['war',
  'military',
  'soviet',
  'president',
  'army',
  'red',
  'vietnam',
  'u.s',
  'american',
  'force'],
 ['series',
  'esk',
  'transformation',
  'century',
  'musil',
  'discworld',
  'magic',
  'eve',
  'dean',
  'various'],
 ['book',
  'read',
  'recommend',
  'story',
  'game',
  'love',
  'star',
  'highly',
  'also',
  'really'],
 ['book',
  'one',
  'good',
  'use',
  'read',
  'many',
  'find',
  'also',
  'make',
  'well'],
 ['....',
  'john',
  'card',
  'real',
  'rich',
  'financial',
  'dirk',
  'company',
  'estate',
  'poor'],
 ['news',
  'human',
  'people',
  'drug',
  'number',
  'state',
  'cancer',
  'society',
  'politic',
  'report'],
 ['plant',
  'page',
  'guide',
  'north',
  'title',
  'common',
  'picture',
  'javascript',
  'design',
  'color'],
 ['human',
  'horse',
  'machine',
  'behavior',
  'whelan',
  'brain',
  'model',
  'von',
  'intelligence',
  'computer'],
 ['theory',
  'evidence',
  'science',
  'study',
  'scientific',
  'claim',
